# EDC-MCP-LLM - Notebook di Test

**Autore:** Lorenzo - Principal Data Architect @ NTT Data Italia

Notebook per testare interattivamente le funzionalita del progetto EDC-MCP-LLM.

**Prerequisiti:**
- Essere nella directory `lineageAI/notebooks/`
- Aver attivato il venv del progetto
- Aver configurato `.env` con le credenziali EDC

---

## 1. Setup e Import

In [1]:
# Setup path per importare i moduli del progetto
import sys
from pathlib import Path

# Aggiungi la root del progetto al path
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
print(f"Python: {sys.executable}")

Project root: c:\Dev\ai-training
Python: c:\Dev\ai-training\lineageAI\.venv\Scripts\python.exe


In [2]:
# Per eseguire codice async in Jupyter
import nest_asyncio
nest_asyncio.apply()

import asyncio
import warnings
warnings.filterwarnings('ignore')

print("[OK] nest_asyncio attivato")

[OK] nest_asyncio attivato


In [3]:
# Import dei moduli del progetto
from src.config.settings import settings, LLMProvider
from src.edc.client import EDCClient
from src.edc.lineage import LineageBuilder
from src.edc.models import TreeNode, LineageDirection
from src.llm.factory import LLMFactory, LLMConfig

print("[OK] Moduli importati")
print(f"    EDC URL: {settings.edc_base_url}")
print(f"    LLM Provider default: {settings.default_llm_provider.value}")

[OK] Settings loaded successfully from .env
   EDC URL: https://edc.servizi.allitude.it:9086/access
[OK] Moduli importati
    EDC URL: https://edc.servizi.allitude.it:9086/access
    LLM Provider default: gemma3


## 2. Inizializzazione Client

In [4]:
# Inizializza EDC Client
edc_client = EDCClient()
print("[OK] EDC Client inizializzato")

# Inizializza LineageBuilder
lineage_builder = LineageBuilder()
print("[OK] LineageBuilder inizializzato")

[OK] EDC Client inizializzato
[OK] LineageBuilder inizializzato


In [9]:
# Inizializza LLM Client
# Puoi cambiare provider: TINYLLAMA, CLAUDE, GEMMA3
CURRENT_LLM_PROVIDER = settings.default_llm_provider

def init_llm_client(provider: LLMProvider = None):
    """Inizializza o cambia il client LLM."""
    global llm_client, CURRENT_LLM_PROVIDER

    if provider:
        CURRENT_LLM_PROVIDER = provider

    if CURRENT_LLM_PROVIDER == LLMProvider.TINYLLAMA:
        config = LLMConfig(
            provider=LLMProvider.TINYLLAMA,
            model_name=settings.tinyllama_model,
            base_url=settings.tinyllama_base_url,
            max_tokens=settings.tinyllama_max_tokens,
            temperature=settings.tinyllama_temperature
        )
    elif CURRENT_LLM_PROVIDER == LLMProvider.GEMMA3:
        config = LLMConfig(
            provider=LLMProvider.GEMMA3,
            model_name=settings.gemma3_model,
            base_url=settings.gemma3_base_url,
            max_tokens=settings.gemma3_max_tokens,
            temperature=settings.gemma3_temperature
        )
    elif CURRENT_LLM_PROVIDER == LLMProvider.CLAUDE:
        config = LLMConfig(
            provider=LLMProvider.CLAUDE,
            model_name=settings.claude_model,
            api_key=settings.claude_api_key,
            max_tokens=settings.claude_max_tokens,
            temperature=settings.claude_temperature
        )
    else:
        raise ValueError(f"Provider non supportato: {CURRENT_LLM_PROVIDER}")

    llm_client = LLMFactory.create_llm_client(config)
    print(f"[OK] LLM Client: {type(llm_client).__name__} ({CURRENT_LLM_PROVIDER.value})")
    return llm_client

llm_client = init_llm_client()

[OK] LLM Client: Gemma3Client (gemma3)


---

## 3. Funzioni di Test

### 3.1 Search Assets

In [ ]:
async def search_assets(
    resource_name: str,
    name_filter: str = "",
    asset_type: str = "com.infa.ldm.relational.Table",
    max_results: int = 10
):
    """
    Cerca asset nel catalogo EDC.

    Args:
        resource_name: Nome della risorsa EDC (es: DataPlatform)
        name_filter: Filtro sul nome (case-insensitive)
        asset_type: Tipo di asset (Table, View, Column, etc.)
        max_results: Numero massimo di risultati
    """
    print(f"\n{'='*60}")
    print(f"SEARCH ASSETS")
    print(f"{'='*60}")
    print(f"Resource: {resource_name}")
    print(f"Filter: {name_filter or '(nessuno)'}")
    print(f"Type: {asset_type}")
    print(f"{'='*60}\n")

    results = await edc_client.bulk_search_assets(
        resource_name=resource_name,
        name_filter=name_filter if name_filter else None,
        asset_type_filter=asset_type
    )

    results = results[:max_results]

    print(f"Trovati {len(results)} asset:\n")

    for i, asset in enumerate(results, 1):
        print(f"{i}. {asset.get('name', 'N/A')}")
        print(f"   Type: {asset.get('classType', 'N/A')}")
        print(f"   ID: {asset.get('id', 'N/A')}")
        print()

    return results

In [10]:
# TEST: Cerca tabelle con GARANZIE nel nome
results = await search_assets(
    resource_name="DataPlatform",
    name_filter="GARANZIE",
    asset_type="com.infa.ldm.relational.Table",
    max_results=10
)

INFO:edc_client:Bulk search: {'resourceName': 'DataPlatform', 'classTypes': 'com.infa.ldm.relational.Table', 'facts': 'id,core.name,core.classType', 'includeRefObjects': 'true'}



SEARCH ASSETS
Resource: DataPlatform
Filter: GARANZIE
Type: com.infa.ldm.relational.Table



INFO:edc_client:Response status: 200
INFO:edc_client:Response Content-Type: application/octet-stream;charset=UTF-8
INFO:edc_client:Received 482756 characters
INFO:edc_client:Parsed 5684 items from CSV
INFO:edc_client:Filtered to 52 items


Trovati 10 asset:

1. STTB_SBS_MIS_COLL_GARANZIE_RAPPORTI_ANACREDIT
   Type: Table
   ID: DataPlatform://ORAC51/DHSTAGE/STTB_SBS_MIS_COLL_GARANZIE_RAPPORTI_ANACREDIT

2. DIM_GARANZIE_STATALI
   Type: Table
   ID: DataPlatform://ORAC51/DATAMARTREP/DIM_GARANZIE_STATALI

3. STTB_SBS_RIPARTO_GARANZIE_SDV
   Type: Table
   ID: DataPlatform://ORAC51/DHSTAGE/STTB_SBS_RIPARTO_GARANZIE_SDV

4. STTB_SID_GARANZIE_INGRESSO_SOFFERENZA_STORICO
   Type: Table
   ID: DataPlatform://ORAC51/DHSTAGE/STTB_SID_GARANZIE_INGRESSO_SOFFERENZA_STORICO

5. STTB_SBS_MIS_GARANZIE_ANACREDIT
   Type: Table
   ID: DataPlatform://ORAC51/DHSTAGE/STTB_SBS_MIS_GARANZIE_ANACREDIT

6. STTB_SID_GARANZIE_INGRESSO_DEFAULT_STORICO
   Type: Table
   ID: DataPlatform://ORAC51/DHSTAGE/STTB_SID_GARANZIE_INGRESSO_DEFAULT_STORICO

7. STTB_SBS_ANAGRAFICA_GARANZIE_SDV
   Type: Table
   ID: DataPlatform://ORAC51/DHSTAGE/STTB_SBS_ANAGRAFICA_GARANZIE_SDV

8. CREWKTBP_GARANZIE_STATALI_M
   Type: Table
   ID: DataPlatform://ORAC51/DWHEVO/C

In [9]:
# TEST: Cerca View
results = await search_assets(
    resource_name="DataPlatform",
    name_filter="GARANZIE",
    asset_type="com.infa.ldm.relational.View",
    max_results=5
)

INFO:edc_client:Bulk search: {'resourceName': 'DataPlatform', 'classTypes': 'com.infa.ldm.relational.View', 'facts': 'id,core.name,core.classType', 'includeRefObjects': 'true'}



SEARCH ASSETS
Resource: DataPlatform
Filter: GARANZIE
Type: com.infa.ldm.relational.View



INFO:edc_client:Response status: 200
INFO:edc_client:Response Content-Type: application/octet-stream;charset=UTF-8
INFO:edc_client:Received 669004 characters
INFO:edc_client:Parsed 7423 items from CSV
INFO:edc_client:Filtered to 121 items


Trovati 5 asset:

1. AMLVW_AS_GARANZIEDIRITTI
   Type: View
   ID: DataPlatform://ORAC51/DATALAYER/AMLVW_AS_GARANZIEDIRITTI

2. AMLVW_AS_CODICIGARANZIE
   Type: View
   ID: DataPlatform://ORAC51/DATALAYER/AMLVW_AS_CODICIGARANZIE

3. AMLVW_AS_GARANZIE
   Type: View
   ID: DataPlatform://ORAC51/DATALAYER/AMLVW_AS_GARANZIE

4. AMLVW_CHIAVIANACREDITGARANZIE
   Type: View
   ID: DataPlatform://ORAC51/DATALAYER/AMLVW_CHIAVIANACREDITGARANZIE

5. AMLVW_GARANZIE
   Type: View
   ID: DataPlatform://ORAC51/DATALAYER/AMLVW_GARANZIE



### 3.2 Get Asset Details

In [ ]:
async def get_asset_details(asset_id: str, enhance_with_ai: bool = True):
    """
    Recupera dettagli di un asset con enhancement AI opzionale.
    """
    print(f"\n{'='*60}")
    print(f"GET ASSET DETAILS")
    print(f"{'='*60}")
    print(f"Asset: {asset_id}")
    print(f"AI Enhancement: {enhance_with_ai}")
    print(f"{'='*60}\n")

    details = await edc_client.get_asset_details(asset_id)

    print(f"Nome: {details['name']}")
    print(f"Tipo: {details['classType']}")
    print(f"Descrizione: {details['description'] or '(nessuna)'}")
    print(f"\nUpstream links: {len(details['src_links'])}")
    print(f"Downstream links: {len(details['dst_links'])}")
    print(f"Facts: {len(details['facts'])} items")

    if details['src_links']:
        print(f"\nUpstream (prime 5):")
        for link in details['src_links'][:5]:
            print(f"  <- {link['name']} ({link['classType']})")

    if details['dst_links']:
        print(f"\nDownstream (prime 5):")
        for link in details['dst_links'][:5]:
            print(f"  -> {link['name']} ({link['classType']})")

    if enhance_with_ai and llm_client:
        print(f"\n--- AI Enhancement ({CURRENT_LLM_PROVIDER.value}) ---")
        enhanced = await llm_client.enhance_description(
            asset_name=details['name'],
            technical_desc=details['description'],
            schema_context="",
            column_info=[]
        )
        print(f"\n{enhanced}")

    return details

In [ ]:
# TEST: Dettagli asset
asset_id = "DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_SOFFERENZE_DT_AP"

details = await get_asset_details(asset_id, enhance_with_ai=True)

### 3.3 Get Immediate Lineage

In [ ]:
async def get_immediate_lineage(asset_id: str, direction: str = "both"):
    """
    Recupera lineage immediato (1 livello).

    Args:
        asset_id: ID dell'asset
        direction: "upstream", "downstream", "both"
    """
    print(f"\n{'='*60}")
    print(f"GET IMMEDIATE LINEAGE")
    print(f"{'='*60}")
    print(f"Asset: {asset_id}")
    print(f"Direction: {direction}")
    print(f"{'='*60}\n")

    lineage = await lineage_builder.get_immediate_lineage(asset_id, direction)

    upstream = [l for l in lineage if l['direction'] == 'upstream']
    downstream = [l for l in lineage if l['direction'] == 'downstream']

    if direction in ['upstream', 'both']:
        print(f"UPSTREAM: {len(upstream)} asset")
        print("-" * 40)
        for link in upstream:
            print(f"  <- {link['name']}")
            print(f"     Type: {link['classType']}")
            print(f"     ID: {link['asset_id']}")
            print()

    if direction in ['downstream', 'both']:
        print(f"\nDOWNSTREAM: {len(downstream)} asset")
        print("-" * 40)
        for link in downstream:
            print(f"  -> {link['name']}")
            print(f"     Type: {link['classType']}")
            print(f"     ID: {link['asset_id']}")
            print()

    return lineage

In [ ]:
# TEST: Lineage immediato
asset_id = "DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_SOFFERENZE_DT_AP"

lineage = await get_immediate_lineage(asset_id, direction="both")

### 3.4 Build Lineage Tree

In [ ]:
def print_tree(node: TreeNode, indent: int = 0):
    """Stampa albero in modo leggibile."""
    prefix = "  " * indent
    type_short = node.class_type.split('.')[-1] if '.' in node.class_type else node.class_type
    print(f"{prefix}[{node.code}] {node.name} ({type_short})")
    for child in node.children:
        print_tree(child, indent + 1)


async def build_lineage_tree(asset_id: str, max_depth: int = 3):
    """
    Costruisce albero di lineage upstream.
    """
    print(f"\n{'='*60}")
    print(f"BUILD LINEAGE TREE")
    print(f"{'='*60}")
    print(f"Asset: {asset_id}")
    print(f"Max Depth: {max_depth}")
    print(f"{'='*60}\n")

    import time
    start = time.time()

    # Pulisci cache per evitare cicli da run precedenti
    lineage_builder.clear_cache()

    root = await lineage_builder.build_tree(
        node_id=asset_id,
        code="001",
        depth=0,
        max_depth=max_depth
    )

    elapsed = time.time() - start

    if root:
        stats = root.get_statistics()
        print(f"Costruito in {elapsed:.2f}s")
        print(f"\nStatistiche:")
        print(f"  - Nodi totali: {stats['total_nodes']}")
        print(f"  - Profondita max: {stats['max_depth']}")
        print(f"  - Nodi terminali: {stats['terminal_nodes']}")
        print(f"\nStruttura albero:\n")
        print_tree(root)
    else:
        print("Nessun lineage trovato.")

    return root

In [ ]:
# TEST: Costruisci albero lineage
asset_id = "DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_SOFFERENZE_DT_AP"

tree = await build_lineage_tree(asset_id, max_depth=2)

### 3.5 Analyze Change Impact

In [ ]:
async def analyze_change_impact(
    asset_id: str,
    change_type: str,
    change_description: str
):
    """
    Analizza l'impatto di una modifica usando AI.

    Args:
        asset_id: Asset da modificare
        change_type: column_drop, data_type_change, deprecation, schema_change, etc.
        change_description: Descrizione della modifica
    """
    print(f"\n{'='*60}")
    print(f"ANALYZE CHANGE IMPACT")
    print(f"{'='*60}")
    print(f"Asset: {asset_id}")
    print(f"Change Type: {change_type}")
    print(f"Description: {change_description}")
    print(f"LLM: {CURRENT_LLM_PROVIDER.value}")
    print(f"{'='*60}\n")

    # Recupera lineage downstream
    lineage = await lineage_builder.get_immediate_lineage(asset_id, "downstream")
    downstream = [l for l in lineage if l['direction'] == 'downstream']

    print(f"Asset downstream impattati: {len(downstream)}")

    # Analisi AI
    impact = await llm_client.analyze_change_impact(
        source_asset=asset_id,
        change_type=change_type,
        change_details={'description': change_description},
        affected_lineage={'downstream': downstream}
    )

    print(f"\n{'='*40}")
    print(f"RISULTATO ANALISI")
    print(f"{'='*40}")
    print(f"\nLIVELLO RISCHIO: {impact['risk_level']}")

    print(f"\nImpatto Business:")
    print(f"  {impact['business_impact']}")

    print(f"\nImpatto Tecnico:")
    print(f"  {impact['technical_impact']}")

    print(f"\nRaccomandazioni:")
    for i, rec in enumerate(impact.get('recommendations', []), 1):
        print(f"  {i}. {rec}")

    print(f"\nStrategia Testing:")
    for i, test in enumerate(impact.get('testing_strategy', []), 1):
        print(f"  {i}. {test}")

    return impact

In [ ]:
# TEST: Analisi impatto
impact = await analyze_change_impact(
    asset_id="DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_SOFFERENZE_DT_AP",
    change_type="column_drop",
    change_description="Eliminazione colonna FLAG_ATTIVO per deprecazione"
)

### 3.6 Generate Change Checklist

In [ ]:
async def generate_change_checklist(
    asset_id: str,
    change_type: str,
    change_description: str
):
    """
    Genera checklist operativa per una modifica.
    """
    print(f"\n{'='*60}")
    print(f"GENERATE CHANGE CHECKLIST")
    print(f"{'='*60}")
    print(f"Asset: {asset_id}")
    print(f"Modifica: {change_type} - {change_description}")
    print(f"{'='*60}\n")

    # Prima genera impact analysis
    lineage = await lineage_builder.get_immediate_lineage(asset_id, "downstream")
    downstream = [l for l in lineage if l['direction'] == 'downstream']

    impact = await llm_client.analyze_change_impact(
        source_asset=asset_id,
        change_type=change_type,
        change_details={'description': change_description},
        affected_lineage={'downstream': downstream}
    )

    # Genera checklist
    checklist = await llm_client.generate_change_checklist(impact)

    sections = [
        ('governance_tasks', 'GOVERNANCE E APPROVAZIONI'),
        ('pre_change_tasks', 'PREPARAZIONE PRE-MODIFICA'),
        ('execution_tasks', 'ESECUZIONE'),
        ('validation_tasks', 'VALIDAZIONE E TEST'),
        ('rollback_procedures', 'PROCEDURE DI ROLLBACK'),
        ('stakeholder_notifications', 'NOTIFICHE STAKEHOLDER'),
        ('monitoring_tasks', 'MONITORING POST-IMPLEMENTAZIONE')
    ]

    print("CHECKLIST OPERATIVA")
    print("=" * 40)

    for key, title in sections:
        tasks = checklist.get(key, [])
        if tasks:
            print(f"\n{title}:")
            for i, task in enumerate(tasks, 1):
                print(f"  [ ] {i}. {task}")

    return checklist

In [ ]:
# TEST: Genera checklist
checklist = await generate_change_checklist(
    asset_id="DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_SOFFERENZE_DT_AP",
    change_type="schema_change",
    change_description="Aggiunta nuova colonna DATA_SCADENZA"
)

### 3.7 Enhance Asset Documentation

In [ ]:
async def enhance_asset_documentation(asset_id: str, business_domain: str = ""):
    """
    Arricchisce la documentazione di un asset con AI.
    """
    print(f"\n{'='*60}")
    print(f"ENHANCE ASSET DOCUMENTATION")
    print(f"{'='*60}")
    print(f"Asset: {asset_id}")
    print(f"Domain: {business_domain or '(auto)'}")
    print(f"LLM: {CURRENT_LLM_PROVIDER.value}")
    print(f"{'='*60}\n")

    # Recupera metadati
    metadata = await lineage_builder.get_asset_metadata(asset_id)

    print(f"Asset: {metadata['name']}")
    print(f"Tipo: {metadata['classType']}")
    print(f"Descrizione originale: {metadata['description'] or '(nessuna)'}")

    # Recupera lineage per contesto
    lineage = await lineage_builder.get_immediate_lineage(asset_id, "upstream")
    upstream = [l for l in lineage if l['direction'] == 'upstream']

    # Enhancement AI
    enhanced = await llm_client.enhance_documentation(
        asset_info=metadata,
        lineage_context={'upstream': upstream},
        business_context={'domain': business_domain}
    )

    print(f"\n{'='*40}")
    print(f"DOCUMENTAZIONE ARRICCHITA")
    print(f"{'='*40}")

    print(f"\nDescrizione Arricchita:")
    print(f"  {enhanced['enhanced_description']}")

    print(f"\nScopo Business:")
    print(f"  {enhanced['business_purpose']}")

    print(f"\nTag Suggeriti:")
    print(f"  {', '.join(enhanced['suggested_tags'])}")

    print(f"\nRegole Qualita Suggerite:")
    for i, rule in enumerate(enhanced.get('suggested_quality_rules', []), 1):
        print(f"  {i}. {rule}")

    print(f"\nNote Compliance:")
    print(f"  {enhanced['compliance_notes']}")

    return enhanced

In [ ]:
# TEST: Arricchisci documentazione
enhanced = await enhance_asset_documentation(
    asset_id="DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_SOFFERENZE_DT_AP",
    business_domain="GARANZIE"
)

### 3.8 Test MCP Prompts

Questa sezione testa il sistema di Prompts MCP integrato nel server.
I prompts sono template pre-configurati che orchestrano automaticamente i tools.

In [11]:
# Per testare i prompts, dobbiamo simulare le chiamate che Claude Desktop fa al server MCP
from src.mcp.server import EDCMCPServer
from mcp.types import GetPromptResult

print("[INFO] Inizializzazione MCP Server per test prompts...")

# Crea istanza server (non avviarlo, solo per testare i metodi)
try:
    mcp_server = EDCMCPServer()
    print("[OK] MCP Server inizializzato per testing")
except Exception as e:
    print(f"[ERROR] Impossibile inizializzare MCP Server: {e}")
    print("[INFO] Assicurati di aver applicato le patch per i prompts")
    mcp_server = None

[INFO] Inizializzazione MCP Server per test prompts...
[OK] MCP Server inizializzato per testing


[MCP] ===== INIZIO INIZIALIZZAZIONE =====
[MCP] Python version: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
[MCP] Working directory: c:\Dev\ai-training\lineageAI
[MCP] [OK] Server MCP creato
[MCP] [OK] Provider LLM: gemma3
[MCP] Inizializzazione LLM client...
[MCP] _initialize_llm: provider=gemma3
[MCP] Configurazione Gemma3...
[MCP] Config: model=gemma3:4b, url=http://localhost:11434
[MCP] Creazione client LLM tramite factory...
[MCP] [OK] Client creato: Gemma3Client
[MCP] [OK] LLM client inizializzato: Gemma3Client
[MCP] Registrazione tools e prompts...
[MCP] Inizio registrazione decoratori...
[MCP] [OK] Decoratore list_tools registrato
[MCP] [OK] Decoratore list_prompts registrato
[MCP] [OK] Decoratore get_prompt registrato
[MCP] [OK] Decoratore call_tool registrato
[MCP] [OK] Registrazione tools e prompts completata
[MCP] [OK] Tools e prompts registrati
[MCP] ===== INIZIALIZZAZIONE COMPLETATA =====


In [12]:
async def list_available_prompts():
    """Lista tutti i prompts MCP disponibili."""
    if not mcp_server:
        print("[ERROR] MCP Server non disponibile")
        return []

    print(f"\n{'='*60}")
    print("LISTA PROMPTS MCP DISPONIBILI")
    print(f"{'='*60}\n")

    prompts_info = [
        {
            "name": "analyze_asset_comprehensive",
            "description": "Analisi completa di un asset EDC con contesto business, tecnico e governance",
            "arguments": [
                {"name": "asset_id", "required": True},
                {"name": "include_lineage", "required": False},
                {"name": "business_domain", "required": False}
            ]
        },
        {
            "name": "impact_analysis_template",
            "description": "Template guidato per analisi impatto di una modifica",
            "arguments": [
                {"name": "asset_id", "required": True},
                {"name": "change_type", "required": True},
                {"name": "change_description", "required": True}
            ]
        },
        {
            "name": "data_governance_review",
            "description": "Revisione governance per un asset",
            "arguments": [
                {"name": "asset_id", "required": True},
                {"name": "review_type", "required": False}
            ]
        },
        {
            "name": "lineage_investigation",
            "description": "Investigazione approfondita del lineage",
            "arguments": [
                {"name": "asset_id", "required": True},
                {"name": "investigation_goal", "required": True},
                {"name": "depth", "required": False}
            ]
        },
        {
            "name": "migration_planning",
            "description": "Pianificazione migrazione di un asset",
            "arguments": [
                {"name": "asset_id", "required": True},
                {"name": "target_environment", "required": True},
                {"name": "migration_type", "required": False}
            ]
        },
        {
            "name": "documentation_enhancement",
            "description": "Arricchimento documentazione",
            "arguments": [
                {"name": "asset_id", "required": True},
                {"name": "documentation_level", "required": False}
            ]
        }
    ]

    for i, prompt in enumerate(prompts_info, 1):
        print(f"{i}. {prompt['name']}")
        print(f"   {prompt['description']}")
        req_args = [a['name'] for a in prompt['arguments'] if a['required']]
        opt_args = [a['name'] for a in prompt['arguments'] if not a['required']]
        if req_args:
            print(f"   Obbligatori: {', '.join(req_args)}")
        if opt_args:
            print(f"   Opzionali: {', '.join(opt_args)}")
        print()

    print(f"{'='*60}")
    print(f"Totale: {len(prompts_info)} prompts")

    return prompts_info

In [13]:
# TEST: Lista prompts disponibili
prompts = await list_available_prompts()


LISTA PROMPTS MCP DISPONIBILI

1. analyze_asset_comprehensive
   Analisi completa di un asset EDC con contesto business, tecnico e governance
   Obbligatori: asset_id
   Opzionali: include_lineage, business_domain

2. impact_analysis_template
   Template guidato per analisi impatto di una modifica
   Obbligatori: asset_id, change_type, change_description

3. data_governance_review
   Revisione governance per un asset
   Obbligatori: asset_id
   Opzionali: review_type

4. lineage_investigation
   Investigazione approfondita del lineage
   Obbligatori: asset_id, investigation_goal
   Opzionali: depth

5. migration_planning
   Pianificazione migrazione di un asset
   Obbligatori: asset_id, target_environment
   Opzionali: migration_type

6. documentation_enhancement
   Arricchimento documentazione
   Obbligatori: asset_id
   Opzionali: documentation_level

Totale: 6 prompts


In [14]:
async def get_prompt_template(prompt_name: str, arguments: dict):
    """Recupera un prompt specifico con argomenti compilati."""
    if not mcp_server:
        print("[ERROR] MCP Server non disponibile")
        return None

    print(f"\n{'='*60}")
    print(f"GET PROMPT: {prompt_name}")
    print(f"{'='*60}")
    print(f"Argomenti: {arguments}\n")

    try:
        # Chiama il metodo generator del server
        generators = {
            "analyze_asset_comprehensive": mcp_server._generate_analyze_asset_prompt,
            "impact_analysis_template": mcp_server._generate_impact_analysis_prompt,
            "data_governance_review": mcp_server._generate_governance_review_prompt,
            "lineage_investigation": mcp_server._generate_lineage_investigation_prompt,
            "migration_planning": mcp_server._generate_migration_planning_prompt,
            "documentation_enhancement": mcp_server._generate_documentation_enhancement_prompt
        }

        if prompt_name not in generators:
            print(f"[ERROR] Prompt '{prompt_name}' non riconosciuto")
            return None

        result = await generators[prompt_name](arguments)

        print(f"[OK] Prompt generato\n")
        print(f"Descrizione: {result.description}\n")
        print("Contenuto prompt:")
        print("-" * 60)

        for message in result.messages:
            content = message.content.text
            # Mostra primi 800 caratteri
            if len(content) > 800:
                print(f"{content[:800]}...\n[... {len(content)-800} caratteri rimanenti ...]")
            else:
                print(content)

        print("-" * 60)
        print(f"\n[INFO] In Claude Desktop, questo prompt orchestrera' automaticamente")
        print("       i tools MCP necessari per generare l'analisi completa.")

        return result

    except Exception as e:
        print(f"[ERROR] Errore generazione prompt: {e}")
        import traceback
        traceback.print_exc()
        return None

In [15]:
# TEST 1: Prompt analyze_asset_comprehensive
prompt_result = await get_prompt_template(
    prompt_name="analyze_asset_comprehensive",
    arguments={
        "asset_id": "DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_DT",
        "include_lineage": True,
        "business_domain": "GARANZIE"
    }
)


GET PROMPT: analyze_asset_comprehensive
Argomenti: {'asset_id': 'DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_DT', 'include_lineage': True, 'business_domain': 'GARANZIE'}

[OK] Prompt generato

Descrizione: Analisi completa dell'asset DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_DT

Contenuto prompt:
------------------------------------------------------------
Analizza in modo completo questo asset del catalogo EDC - usa i tools MCP disponibili.
------------------------------------------------------------

[INFO] In Claude Desktop, questo prompt orchestrera' automaticamente
       i tools MCP necessari per generare l'analisi completa.


[MCP] >> Generazione prompt analyze_asset_comprehensive per DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_DT


In [ ]:
# TEST 2: Prompt impact_analysis_template
prompt_result = await get_prompt_template(
    prompt_name="impact_analysis_template",
    arguments={
        "asset_id": "DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_SOFFERENZE_DT_AP",
        "change_type": "column_drop",
        "change_description": "Eliminazione colonna FLAG_OBSOLETO non piu utilizzata"
    }
)

In [ ]:
# TEST 3: Prompt data_governance_review
prompt_result = await get_prompt_template(
    prompt_name="data_governance_review",
    arguments={
        "asset_id": "DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_DT",
        "review_type": "complete"
    }
)

### Come Usare i Prompts in Claude Desktop

Una volta applicata la patch al server e riavviato Claude Desktop:

```
Usa il prompt "analyze_asset_comprehensive" per analizzare
l'asset DataPlatform://ORAC51/DWHEVO/IFR_WK_GARANZIE_DT
includendo il lineage e il dominio GARANZIE
```

**Claude Desktop:**
1. Lista i prompts disponibili tramite `list_prompts()`
2. Recupera il prompt compilato tramite `get_prompt()`
3. Esegue automaticamente i tools MCP necessari
4. Genera l'analisi completa strutturata

**Vantaggi dei Prompts:**
- Template pre-configurati per task complessi
- Orchestrazione automatica di piu tools
- Output strutturati e consistenti
- Workflow guidati per governance e change management

**Differenza Tools vs Prompts:**

| Tools | Prompts |
|-------|--------|
| Azioni singole | Workflow completi |
| `search_assets` | "analizza completamente questo asset" |
| Devi sapere quale tool usare | Template guidati |
| Query dirette | Analisi strutturate |


---

## 4. Utility

In [16]:
def show_statistics():
    """Mostra statistiche del sistema."""
    print(f"\n{'='*60}")
    print(f"STATISTICHE SISTEMA")
    print(f"{'='*60}")

    stats = lineage_builder.get_statistics()

    print(f"\nEDC Client:")
    print(f"  - Total API calls: {stats['total_requests']}")
    print(f"  - Cache hits: {stats['cache_hits']}")
    print(f"  - API errors: {stats['api_errors']}")
    print(f"  - Nodi creati: {stats['nodes_created']}")
    print(f"  - Cicli prevenuti: {stats['cycles_prevented']}")

    if stats['total_requests'] > 0:
        cache_rate = stats['cache_hits'] / stats['total_requests'] * 100
        print(f"  - Cache hit rate: {cache_rate:.1f}%")

    print(f"\nLLM Provider: {CURRENT_LLM_PROVIDER.value}")


show_statistics()


STATISTICHE SISTEMA

EDC Client:
  - Total API calls: 0
  - Cache hits: 0
  - API errors: 0
  - Nodi creati: 0
  - Cicli prevenuti: 0

LLM Provider: gemma3


In [ ]:
def switch_llm(provider: str):
    """
    Cambia provider LLM.

    Args:
        provider: "tinyllama", "claude", "gemma3"
    """
    provider_map = {
        'tinyllama': LLMProvider.TINYLLAMA,
        'claude': LLMProvider.CLAUDE,
        'gemma3': LLMProvider.GEMMA3
    }

    if provider.lower() not in provider_map:
        print(f"Provider non valido. Usa: {list(provider_map.keys())}")
        return

    init_llm_client(provider_map[provider.lower()])


# Esempio: switch_llm("claude")

In [ ]:
def clear_cache():
    """Pulisce la cache."""
    lineage_builder.clear_cache()
    print("[OK] Cache pulita")


# clear_cache()

---

## 5. Cleanup

In [ ]:
# Chiudi connessioni alla fine
async def cleanup():
    await lineage_builder.close()
    print("[OK] Connessioni chiuse")

# Decommentare per chiudere:
# await cleanup()

---

## Note

### Asset ID Format
```
DataPlatform://CONNECTION/SCHEMA/TABLE_NAME
```

### Provider LLM disponibili
- `tinyllama` - Via Ollama locale (veloce, meno preciso)
- `gemma3` - Via Ollama locale (bilanciato)
- `claude` - Anthropic API (preciso, richiede API key)

### Cambiare provider
```python
switch_llm("claude")  # o "tinyllama" o "gemma3"
```